In [1]:
import sys
sys.path.append('../')

In [2]:
from trading_env import DataSource, TradeBroker, TradingEnv
import trading_env

In [3]:
d = DataSource("../../Data/DAT_MT_EURUSD_M1_2021.csv", end_index=30)
tb = TradeBroker(d, initial_balance=10, price_history_length=5)
env = TradingEnv(tb)

In [4]:
env.reset()

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [5]:
for i in range(40):
    obs, reward, done, info = env.step(2)
    if done is True: 
        break

2024-03-08 00:41:40,915 - trading_env - DEBUG - Holding at 1.22322
2024-03-08 00:41:40,918 - trading_env - DEBUG - Holding at 1.22315
2024-03-08 00:41:40,921 - trading_env - DEBUG - Holding at 1.22356
2024-03-08 00:41:40,924 - trading_env - DEBUG - Holding at 1.22341
2024-03-08 00:41:40,926 - trading_env - DEBUG - Holding at 1.2237
2024-03-08 00:41:40,929 - trading_env - DEBUG - Holding at 1.2236
2024-03-08 00:41:40,932 - trading_env - DEBUG - Holding at 1.22354
2024-03-08 00:41:40,934 - trading_env - DEBUG - Holding at 1.22339
2024-03-08 00:41:40,936 - trading_env - DEBUG - Holding at 1.22343
2024-03-08 00:41:40,939 - trading_env - DEBUG - Holding at 1.22333
2024-03-08 00:41:40,941 - trading_env - DEBUG - Holding at 1.22333
2024-03-08 00:41:40,943 - trading_env - DEBUG - Holding at 1.22333
2024-03-08 00:41:40,946 - trading_env - DEBUG - Holding at 1.22366
2024-03-08 00:41:40,948 - trading_env - DEBUG - Holding at 1.22392
2024-03-08 00:41:40,950 - trading_env - DEBUG - Holding at 1.223

In [6]:
import random

In [7]:
d = DataSource("../../Data/DAT_MT_EURUSD_M1_2021.csv",)
tb = TradeBroker(d, initial_balance=1000, price_history_length=10)
env = TradingEnv(tb)
env.reset()
trading_env.logger.setLevel("CRITICAL")
from tqdm import tqdm
total_reward = 0
for i in tqdm(range(20000)):
    obs, reward, done, info = env.step(random.choice([0,1,2]))
    total_reward += reward
    if done is True: 
        break
    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [00:28<00:00, 709.20it/s]


In [8]:
info

{'action': 2,
 'action_res': True,
 'assets': {'balance': 939.0414799999978,
  'shares_held': 48,
  'current_price': 1.18879,
  'P/L': -60.95852000000218}}

In [9]:
total_reward

-60.95852000000218

In [ ]:
import matplotlib

In [11]:
tb.balance_history

[1000,
 1000,
 1000,
 998.77659,
 998.77659,
 1000.0001900000001,
 1000.0001900000001,
 998.7768000000001,
 997.5533700000001,
 997.5533700000001,
 998.7767000000001,
 997.5533700000001,
 996.3297100000001,
 995.1057900000001,
 996.3293900000001,
 996.3293900000001,
 997.5533700000001,
 996.3290400000001,
 997.55352,
 996.3288200000001,
 996.3288200000001,
 995.10382,
 996.3288200000001,
 995.1045100000001,
 995.1045100000001,
 993.8799700000001,
 995.10441,
 993.8800600000001,
 995.1046000000001,
 995.1046000000001,
 996.3288200000001,
 996.3288200000001,
 996.3288200000001,
 997.55295,
 997.55295,
 998.77711,
 997.55221,
 996.32745,
 995.10291,
 996.32763,
 995.10318,
 996.3273499999999,
 996.3273499999999,
 997.5523699999999,
 997.5523699999999,
 997.5523699999999,
 996.3271199999999,
 995.1016199999999,
 993.8759099999999,
 992.6503099999999,
 993.8760099999998,
 992.6508699999998,
 991.4254099999998,
 992.6509899999999,
 992.6509899999999,
 991.4257899999999,
 992.6511299999999,
 